In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the IMDB dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

# Pad sequences to ensure all reviews are the same length
X_train = pad_sequences(X_train, maxlen=200)
X_test = pad_sequences(X_test, maxlen=200)

# Convert to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [3]:
from tensorflow.keras import models, layers

def build_model():
    model = models.Sequential()
    model.add(layers.Embedding(input_dim=10000, output_dim=128))  # Removed input_length
    model.add(layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


In [4]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.6893 - loss: 0.5705 - val_accuracy: 0.8174 - val_loss: 0.4123
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - accuracy: 0.8643 - loss: 0.3353 - val_accuracy: 0.8374 - val_loss: 0.3844
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.8865 - loss: 0.2888 - val_accuracy: 0.7818 - val_loss: 0.4494
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9067 - loss: 0.2423 - val_accuracy: 0.8340 - val_loss: 0.4103
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.9051 - loss: 0.2415 - val_accuracy: 0.7692 - val_loss: 0.4944
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8765 - loss: 0.2989 - val_accuracy: 0.8288 - val_loss: 0.4408
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.9320 - loss: 0.1795 - val_accuracy: 0.8384 - val_loss: 0.4155
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.9509 - loss: 0.1367 - 

In [5]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8341 - loss: 0.5637
Accuracy: 83.57999920845032


In [6]:
import json

word_index = imdb.get_word_index()

def decode_review(encoded_review):
    reverse_word_index = {value: key for (key, value) in word_index.items()}
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])
    return decoded_review

# Decode the first 3 reviews in the test set
sample_reviews = [decode_review(X_test[i]) for i in range(3)]

# Preprocess reviews
def preprocess_reviews(reviews):
    sequences = imdb.get_word_index()
    indices = [[sequences.get(word, 0) for word in review.split()] for review in reviews]
    return pad_sequences(indices, maxlen=200)

preprocessed_reviews = preprocess_reviews(sample_reviews)


In [7]:
predictions = model.predict(preprocessed_reviews)

# Output the predictions
for review, prediction in zip(sample_reviews, predictions):
    sentiment = 'Positive' if prediction > 0.5 else 'Negative'
    print(f'Review: "{review}"\nPredicted Sentiment: {sentiment}\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
Review: "? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss"
Predicted Sentiment: Positive

Review: "psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker chara